# `autograd`를 이용한 최적화

딥러닝과 같은 머신러닝 모형을 학습한다는건 데이터를 입력하여 파라메터를 목적함수를 만족하는 방향으로 조정하는 과정이다. 딥러닝에서는 이 목적함수가 손실함수(loss function)이라고 일컬어지는데, 정답과 예측값의 차이를 계산하는 함수라고 이해하면 된다. 이 차이가 적을수록 우리가 원하는 모형에 가까워진다고 이야기 할 수 있을 것이다.

고등학교 수학에서 배웠듯이 이러한 목적함수를 최소화 하는 최적화의 과정은 주로 미분을 통해 이루어진다. 미분은 최소, 최대인 점을 알려주기도 하지만 무엇보다 어떻게 최대, 최소의 점에 도달할 수 있는지 정보를 준다.

딥러닝에서 이러한 미분을 이용해서 학습을 하는 과정을 스토케스틱 경사 하강법(stochastic gradient descent)이라한다.

Gluon에서는 `autograd`함수를 이용해서 미분을 자동으로 계산해 준다.

$y = x^2 + 5$일때  $\frac{\partial y}{\partial x}=2x$가 될 것이라 알수 있는데, Gluon에서는 아래와 같은 과정을 통해 계산해 준다.

## Basic usage

`autograd` package를 임포트한다.

In [7]:
from mxnet import nd
from mxnet import autograd
import mxnet as mx

### Autograd for $f(x) =x^2 + 5$

In [8]:
x = nd.arange(0,5,0.5)
x.attach_grad()
with autograd.record():
    y =x**2  + 5
y.backward()
x.grad


[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
<NDArray 10 @cpu(0)>

미분이 계산되어야 되는 영역은 `autograd.record()`를 사용해 `with`문으로 감싸주어야 된다.

그럼 미분이 어떻게 활용되는지 대력적인 학습과정을 예를 들어 살펴보자.

1. 배치(batch)라고 불리우는 일정 수의 학습셋(x, y)을 모형에 입력한다.
2. x 데이터를 입력해 예측값이 pred_y를 도출한다.
3. 실제 정답인 y와 pred_y의 차이 즉 로스(loss)를 계산한다.
4. loss가 0에 수렴할 수 있도록 모형의 파라메터를 조정한다.


이 4에 해당되는 부분이 미분이 필요한 과정인데, 아래 수식에서와 같이 연쇄적으로 일어난다.

학습할 파라메터가 $\theta = \{W_1, W_2, ... , W_k\}$ 일때 $J(W_1, ...,W_k) = \frac{1}{2} \lVert y - f_{W_k}(...f_{W_{2}}(f_{W_1}(x))) \rVert_2^2$와 같이 표현되는 평균오차(mean sqared error)를 최소화 하는 해를 구하는 과정이라 할 수 있다.

여기서 과연 $W_k$의 학습 조정치인 $\Delta_k$를 구하고 이보다 더 나아가 $\Delta_{k-1}$ 그리고 $\Delta_1$까지의 값을 구해야 되는데 $\Delta_k=\frac{\partial J}{\partial W_k}$와 같은 미분으로 시작해 연쇄적으로 이루어지게 된다.  

다시 간단한 슈도코드(pseudocode)로 나타내면 아래와 같다.

> 1. 랜덤하게 $\theta$에 값을 할당한다.
> 2. repeat until(모든 데이터 순회)
> 3. &nbsp;&nbsp;&nbsp;&nbsp;배치에 포함되는 x에 그레디언트 $\Delta_1, ... , \Delta_{n-1}, \Delta_n$를 구한다.  
> 4. &nbsp;&nbsp;&nbsp;&nbsp;$\Delta_{total}=\frac{1}{n}\sum_{t=1}^n\Delta_t$ # 그레디언트 평균
 > 5. &nbsp;&nbsp;&nbsp;&nbsp;$\theta = \theta - \rho\Delta_{total}$
 > 6. $\hat \theta =\theta$

n은 배치 사이즈이고 모든 데이터를 순회했을때 1에폭(epoch) 학습이 끝났다고 이야기한다.